# ReadtheDocs Retrieval Augmented Generation (RAG) using Milvus Docker Container

In this notebook, we are going to use Milvus documentation pages to create a chatbot about our product.  The chatbot is going to follow RAG steps to retrieve chunks of data using Semantic Vector Search, then the Question + Context will be fed as a Prompt to a LLM to generate an answer.

Many RAG demos use OpenAI for the Embedding Model and ChatGPT for the Generative AI model.  **In this notebook, we will demo a fully open source RAG stack.**

Using open-source Q&A with retrieval saves money since we make free calls to our own data almost all the time - retrieval, evaluation, and development iterations.  We only make a paid call to OpenAI once for the final chat generation step. 

<div>
<img src="../../../images/rag_image.png" width="80%"/>
</div>

Let's get started!

In [1]:
# For colab install these libraries in this order:
# !python -m pip install torch transformers sentence-transformers langchain
# !python -m pip install -U pymilvus 'pymilvus[model]'
# !python -m pip install unstructured openai tqdm numpy ipykernel 
# !python -m pip install ragas datasets

In [2]:
# Import common libraries.
import sys, os, time, pprint
import numpy as np

## Get and save API keys.

The services I'm going to use are:
- [Zilliz Cloud](https://cloud.zilliz.com/login)
- [OpenAI](https://platform.openai.com/api-keys)
- [Anthropic]()
- [Anyscale endpoints]()

💡 Note: To keep your tokens private, best practice is to use an **env variable**.  See [how to save api key in env variable](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety). <br>

👉🏼 In Jupyter, you need a .env file (in same dir as notebooks) containing lines like this:
- ZILLIZ_API_KEY=f370c...
- OPENAI_API_KEY=sk-H...
- VARIABLE_NAME=value...

## Download Data

The data used in this notebook is Milvus documentation web pages.

The code block below downloads all the web pages into a local directory called `rtdocs`.  

I've already uploaded the `rtdocs` data folder to github, so you should see it if you cloned my repo.

In [3]:
# UNCOMMENT TO DOWNLOAD THE DOCS.

# # !pip install -U langchain
# from langchain_community.document_loaders import RecursiveUrlLoader

# DOCS_PAGE="https://milvus.io/docs/"

# loader = RecursiveUrlLoader(DOCS_PAGE)
# docs = loader.load()

# num_documents = len(docs)
# print(f"loaded {num_documents} documents")

In [4]:
# UNCOMMENT TO READ THE DOCS FROM A LOCAL DIRECTORY.

# Read docs into LangChain
# !pip install -U langchain
# !pip install unstructured
from langchain.document_loaders import DirectoryLoader

# Load HTML files from a local directory
path = "../../RAG/rtdocs/"
loader = DirectoryLoader(path, glob='*.html')
docs = loader.load()

num_documents = len(docs)
print(f"loaded {num_documents} documents")

loaded 22 documents


## Start up Milvus running in local Docker (or Zilliz free tier)

>⛔️ Make sure you pip install the correct version of pymilvus and server yml file.  **Versions (major and minor) should all match**.

1. [Install Docker](https://docs.docker.com/get-docker/)
2. Start your Docker Desktop
3. If you have an old Milvus container, previous version, delete it from Docker Desktop.
4. Download the latest [docker-compose.yml](https://milvus.io/docs/install_standalone-docker.md#Download-the-YAML-file) (or run the wget command, replacing version to what you are using)
> wget https://github.com/milvus-io/milvus/releases/download/v2.4.1/milvus-standalone-docker-compose.yml -O docker-compose.yml
5. From your terminal:  
   - cd into directory where you saved the .yml file (usualy same dir as this notebook)
   - docker compose up -d
   - verify (either in terminal or on Docker Desktop) the containers are running
6. From your code (see notebook code below):
   - You already did this (pip install -U pymilvus)
   - Import milvus and connect to the local milvus server

In [5]:
# !pip install -U pymilvus
# !wget https://github.com/milvus-io/milvus/releases/download/v2.4.1/milvus-standalone-docker-compose.yml -O docker-compose.yml

In [6]:
# STEP 1. CONNECT TO MILVUS STANDALONE DOCKER.

import pymilvus, time
from pymilvus import (
    MilvusClient, utility, connections,
    FieldSchema, CollectionSchema, DataType, IndexType,
    Collection, AnnSearchRequest, RRFRanker, WeightedRanker
)
print(f"Pymilvus: {pymilvus.__version__}")

# Connect to the local server.
connection = connections.connect(
  alias="default", 
  host='localhost', # or '0.0.0.0' or 'localhost'
  port='19530'
)

# Get server version.
print(utility.get_server_version())

# Use no-schema Milvus client uses flexible json key:value format.
mc = MilvusClient(connections=connection)

Pymilvus: 2.4.1
v2.4.1


# Optionally, use Zilliz free tier cluster
To use fully-managed Milvus on [Ziliz Cloud free trial](https://cloud.zilliz.com/login).  
  1. Choose the default "Starter" option and accept the default Cloud Provider and Region when you create a cluster. 
  2. On the Cluster main page, copy your `API Key` and store it locally in a .env variable.  See [this note](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety) how to do that.
  3. Also on the Cluster main page, copy the `Public Endpoint URI` and store it somewhere convenient.
  4. Jupyter also requires them in a local .env file. <br>
Anywhere in the bootcamp directory, create a .env file
Insert lines like this, substituting your actual API keys for the sample text: <br>
ZILLIZ_API_KEY=f370c <br>
OPENAI_API_KEY=sk-H <br>
ANYSCALE_ENPOINT_KEY=es <br>
ANTHROPIC_API_KEY=sk-an <br>
VARIABLE_NAME=value <br>
Save the .env file <br>

In [7]:
# # STEP 1. CONNECT TO ZILLIZ CLOUD
# import os
# import pymilvus
# print(f"pymilvus version: {pymilvus.__version__}")
# from pymilvus import connections, utility
# TOKEN = os.getenv("ZILLIZ_API_KEY")

# # Connect to Zilliz cloud using endpoint URI and API key TOKEN.
# # TODO change this.
# CLUSTER_ENDPOINT="https://in03-xxxx.api.gcp-us-west1.zillizcloud.com:443"
# CLUSTER_ENDPOINT="https://in03-48a5b11fae525c9.api.gcp-us-west1.zillizcloud.com:443"
# connections.connect(
#   alias='default',
#   #  Public endpoint obtained from Zilliz Cloud
#   uri=CLUSTER_ENDPOINT,
#   # API key or a colon-separated cluster username and password
#   token=TOKEN,
# )

# # Use no-schema Milvus client uses flexible json key:value format.
# # https://milvus.io/docs/using_milvusclient.md
# mc = MilvusClient(
#     uri=CLUSTER_ENDPOINT,
#     # API key or a colon-separated cluster username and password
#     token=TOKEN)

# # Check if the server is ready and get colleciton name.
# print(f"Type of server: {utility.get_server_version()}")

## Load the Embedding Model checkpoint and use it to create vector embeddings

#### What are Embeddings?

Check out [this blog](https://zilliz.com/glossary/vector-embeddings) for an introduction to embeddings.  

An excellent place to start is by selecting an embedding model from the [HuggingFace MTEB Leaderboard](https://huggingface.co/spaces/mteb/leaderboard), sorted descending by the "Retrieval Average'' column since this task is most relevant to RAG. Then, choose the smallest, highest-ranking embedding model. But, Beware!! some models listed are overfit to the training data, so they won't perform on your data as promised.  

Milvus (and Zilliz) only supports tested embedding models that are not overfit.

#### In this notebook, we will use the **open-source BGE-M3** which supports: 
- over 100 languages
- context lengths of up to 8192
- multiple embedding inferences such as dense (semantic), sparse (lexical), and multi-vector Colbert reranking. 

BGE-M3 holds the distinction of being the first embedding model to offer support for all three retrieval methods, achieving state-of-the-art performance on multi-lingual (MIRACL) and cross-lingual (MKQA) benchmark tests.  [Paper](https://arxiv.org/abs/2402.03216), [HuggingFace](https://huggingface.co/BAAI/bge-m3)

**[Milvus](https://github.com/milvus-io/milvus)**, the world's first Open Source Vector Database, plays a vital role in semantic search with scaleable, efficient storage and search for GenerativeAI workflows. Its advanced functionalities include metadata filtering and hybrid search.  Since version 2.4, Milvus has built-in support for BGE M3.

<div>
<img src="../../../images/bge_m3.png" width="80%"/>
</div>

In [8]:
# STEP 2. USE A MILVUS BUILT-IN OPEN SOURCE EMBEDDING MODEL.

from pymilvus.model.hybrid import BGEM3EmbeddingFunction
import torch

# Initialize torch settings
DEVICE = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
print(f"device: {DEVICE}")

# Initialize a Milvus built-in sparse-dense-reranking encoder.
# https://huggingface.co/BAAI/bge-m3
embedding_model = BGEM3EmbeddingFunction(use_fp16=False, device=DEVICE)
EMBEDDING_DIM = embedding_model.dim['dense']
print(f"dense_dim: {EMBEDDING_DIM}")
print(f"sparse_dim: {embedding_model.dim['sparse']}")
print(f"colbert_dim: {embedding_model.dim['colbert_vecs']}")

device: cpu


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

dense_dim: 1024
sparse_dim: 250002
colbert_dim: 1024


## Create a Milvus collection

You can think of a collection in Milvus like a "table" in SQL databases.  The **collection** will contain the 
- **Schema** (or [no-schema Milvus client](https://milvus.io/docs/using_milvusclient.md)).  
💡 You'll need the vector `EMBEDDING_DIM` parameter from your embedding model.
Typical values are:
   - 1024 for sbert embedding models
   - 1536 for ada-002 OpenAI embedding models
- **Vector index** for efficient vector search
- **Vector distance metric** for measuring nearest neighbor vectors
- **Consistency level**
In Milvus, transactional consistency is possible; however, according to the [CAP theorem](https://en.wikipedia.org/wiki/CAP_theorem), some latency must be sacrificed. 💡 Searching movie reviews is not mission-critical, so [`eventually`](https://milvus.io/docs/consistency.md) consistent is fine here.

## Add a Vector Index

The vector index determines the vector **search algorithm** used to find the closest vectors in your data to the query a user submits.  

Most vector indexes use different sets of parameters depending on whether the database is:
- **inserting vectors** (creation mode) - vs - 
- **searching vectors** (search mode) 

Scroll down the [docs page](https://milvus.io/docs/index.md) to see a table listing different vector indexes available on Milvus.  For example:
- FLAT - deterministic exhaustive search
- IVF_FLAT or IVF_SQ8 - Hash index (stochastic approximate search)
- HNSW - Graph index (stochastic approximate search)
- AUTOINDEX - OSS or [Zilliz cloud](https://docs.zilliz.com/docs/autoindex-explained) automatic index based on type of GPU, size of data.

Besides a search algorithm, we also need to specify a **distance metric**, that is, a definition of what is considered "close" in vector space.  In the cell below, the [`HNSW`](https://github.com/nmslib/hnswlib/blob/master/ALGO_PARAMS.md) search index is chosen.  Its possible distance metrics are one of:
- L2 - L2-norm
- IP - Dot-product
- COSINE - Angular distance

💡 Most use cases work better with normalized embeddings, in which case L2 is useless (every vector has length=1) and IP and COSINE are the same.  Only choose L2 if you plan to keep your embeddings unnormalized.

In [9]:
# STEP 3. CREATE A NO-SCHEMA MILVUS COLLECTION AND DEFINE THE DATABASE INDEX.
# See docstrings for more information.
# https://github.com/milvus-io/pymilvus/blob/master/examples/hello_hybrid_sparse_dense.py

from pymilvus import MilvusClient

# Set the Milvus collection name.
COLLECTION_NAME = "MilvusDocs"

# Specify the data schema for the new Collection.
MAX_LENGTH = 65535
fields = [
    # Use auto generated id as primary key
    FieldSchema(name="id", dtype=DataType.INT64,
                is_primary=True, auto_id=True, max_length=100),
    FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR),
    FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR,
                dim=EMBEDDING_DIM),
    FieldSchema(name="chunk", dtype=DataType.VARCHAR, max_length=MAX_LENGTH),
    FieldSchema(name="source", dtype=DataType.VARCHAR, max_length=MAX_LENGTH),
    FieldSchema(name="h1", dtype=DataType.VARCHAR, max_length=100),
    FieldSchema(name="h2", dtype=DataType.VARCHAR, max_length=MAX_LENGTH),
]
schema = CollectionSchema(fields, "")

# Check if collection already exists, if so drop it.
has = utility.has_collection(COLLECTION_NAME)
if has:
    drop_result = utility.drop_collection(COLLECTION_NAME)
    print(f"Successfully dropped collection: `{COLLECTION_NAME}`")

# Create the collection.
schema = CollectionSchema(fields, "")
col = Collection(COLLECTION_NAME, schema, consistency_level="Eventually")

# Add custom HNSW search index to the collection.
# M = max number graph connections per layer. Large M = denser graph.
# Choice of M: 4~64, larger M for larger data and larger embedding lengths.
M = 16
# efConstruction = num_candidate_nearest_neighbors per layer. 
# Use Rule of thumb: int. 8~512, efConstruction = M * 2.
efConstruction = M * 2
# Create the search index for local Milvus server.
INDEX_PARAMS = dict({
    'M': M,               
    "efConstruction": efConstruction })

# Create indices for the vector fields. 
# The indices will pre-load data into memory for efficient search.
sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
dense_index = {"index_type": "HNSW", "metric_type": "COSINE", "params": INDEX_PARAMS}
col.create_index("sparse_vector", sparse_index)
col.create_index("dense_vector", dense_index)
col.load()

print(f"Successfully created collection: `{COLLECTION_NAME}`")
# print(mc.describe_collection(COLLECTION_NAME))

Successfully dropped collection: `MilvusDocs`
Successfully created collection: `MilvusDocs`


## HTML Chunking

Before embedding, it is necessary to decide your chunk strategy, chunk size, and chunk overlap.  This section uses:
- **Strategy** = Use markdown header hierarchies.  Keep markdown sections together unless they are too long.
- **Chunk size** = Use the embedding model's parameter `MAX_SEQ_LENGTH`
- **Overlap** = Rule-of-thumb 10-15%
- **Function** = 
  - Langchain's `HTMLHeaderTextSplitter` to split markdown sections.
  - Langchain's `RecursiveCharacterTextSplitter` to split up long reviews recursively.


Notice below, each chunk is grounded with the document source page.  <br>
In addition, header titles are kept together with the chunk of markdown text.

In [10]:
# # STEP 4. PREPARE DATA: CHUNK AND EMBED
# !python -m pip install lxml
from langchain_community.document_transformers import BeautifulSoupTransformer
from langchain.text_splitter import HTMLHeaderTextSplitter, RecursiveCharacterTextSplitter
import numpy as np
import pprint

# Define chunk size 512 and overlap 10% chunk_size.
chunk_size = 512
chunk_overlap = np.round(chunk_size * 0.10, 0)
print(f"chunk_size: {chunk_size}, chunk_overlap: {chunk_overlap}")

# Create an instance of the RecursiveCharacterTextSplitter
child_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap,
    length_function = len,  # using built-in Python len function
)

# Define the headers to split on for the HTMLHeaderTextSplitter
headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
]
# Create an instance of the HTMLHeaderTextSplitter
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# Split the HTML text using the HTMLHeaderTextSplitter.
start_time = time.time()
html_header_splits = []
for doc in docs:
    splits = html_splitter.split_text(doc.page_content)
    for split in splits:
        # Add the source URL and header values to the metadata
        metadata = {}
        new_text = split.page_content
        for header_name, metadata_header_name in headers_to_split_on:
            # Handle exception if h1 does not exist.
            try:
                header_value = new_text.split("¶ ")[0].strip()[:100]
                metadata[header_name] = header_value
            except:
                break
            # Handle exception if h2 does not exist.
            try:
                new_text = new_text.split("¶ ")[1].strip()[:50]
            except:
                break
        split.metadata = {
            **metadata,
            "source": doc.metadata["source"]
        }
        # Add the header to the text
        split.page_content = split.page_content
    html_header_splits.extend(splits)

    # # TODO - Uncomment to save each doc.page_content as a local html file under OUTPUT_DIR.
    # OUTPUT_DIR = "output"
    # # Set filename to first 50 characters of h1 header.
    # filename = doc.metadata["source"].split("/")[-1].split(".")[0][:50]
    # with open(f"{OUTPUT_DIR}/{filename}.html", "w") as f:
    #     f.write(doc.page_content)

# Split the documents further into smaller, recursive chunks.
chunks = child_splitter.split_documents(html_header_splits)

end_time = time.time()
print(f"chunking time: {end_time - start_time}")
print(f"docs: {len(docs)}, split into: {len(html_header_splits)}")
print(f"split into chunks: {len(chunks)}, type: list of {type(chunks[0])}") 

# Inspect a chunk.
print()
print("Looking at a sample chunk...")
print(chunks[0].page_content[:100])
print(chunks[0].metadata)

# # TODO - Uncomment to print child splits with their associated header metadata for debugging.
# print()
# for child in chunks:
#     print(f"Content: {child.page_content}")
#     print(f"Metadata: {child.metadata}")
#     print()

chunk_size: 512, chunk_overlap: 51.0
chunking time: 0.02252483367919922
docs: 22, split into: 22
split into chunks: 304, type: list of <class 'langchain_core.documents.base.Document'>

Looking at a sample chunk...
Why Milvus Docs Tutorials Tools Blog Community Stars0 Try Managed Milvus FREE Search Home v2.4.x Abo
{'h1': 'Why Milvus Docs Tutorials Tools Blog Community Stars0 Try Managed Milvus FREE Search Home v2.4.x Abo', 'source': '../../RAG/rtdocs/quickstart.html'}


In [11]:
# Clean up the metadata urls
for doc in chunks:
    new_url = doc.metadata["source"]
    new_url = new_url.replace("../../RAG/rtdocs", "https://milvus.io/docs")
    new_url = new_url.replace(".html", ".md")
    doc.metadata.update({"source": new_url})

print(chunks[0].page_content[:100])
print(chunks[0].metadata)

Why Milvus Docs Tutorials Tools Blog Community Stars0 Try Managed Milvus FREE Search Home v2.4.x Abo
{'h1': 'Why Milvus Docs Tutorials Tools Blog Community Stars0 Try Managed Milvus FREE Search Home v2.4.x Abo', 'source': 'https://milvus.io/docs/quickstart.md'}


Use the built-in Milvus BGE M3 embedding functions.  The output will be 2 vectors:
- `embeddings['dense'][i]` is a list of numpy arrays, one per chunk. Milvus supports more than 1 dense embedding vector if desired, so i is the ith dense embedding vector.
- `embeddings['sparse'][:, [i]]` is a scipy sparse matrix where each column represents a chunk.

In [12]:
# STEP 5. TRANSFORM CHUNKS INTO VECTORS USING EMBEDDING MODEL INFERENCE.

# BGEM3EmbeddingFunction input is docs as a list of strings.
list_of_strings = [doc.page_content for doc in chunks if hasattr(doc, 'page_content')]

# Embedding inference using the Milvus built-in sparse-dense-reranking encoder.
start_time = time.time()
embeddings = embedding_model(list_of_strings)
end_time = time.time()

print(f"Embedding time for {len(list_of_strings)} chunks: ", end="")
print(f"{np.round(end_time - start_time, 2)} seconds")

# Inference Embeddings: 100%|██████████| 19/19 [00:35<00:00,  1.86s/it]
# Embedding time for 304 chunks: 35.74 seconds

Inference Embeddings: 100%|██████████| 19/19 [00:35<00:00,  1.85s/it]

Embedding time for 304 chunks: 35.11 seconds


## Insert data into Milvus

For each original text chunk, we'll write the sextuplet (`chunk, h1, h2, source, dense_vector, sparse_vector`) into the database.

<div>
<img src="../../../images/db_insert_sparse_dense.png" width="80%"/>
</div>

**The Milvus Client wrapper can only handle loading data from a list of dictionaries.**

Otherwise, in general, Milvus supports loading data from:
- pandas dataframes 
- list of dictionaries

Below, we use the embedding model provided by HuggingFace, download its checkpoint, and run it locally as the encoder.  

In [13]:
# STEP 6. INSERT CHUNK LIST INTO MILVUS OR ZILLIZ.

# Create chunk_list and dict_list in a single loop
dict_list = []
for chunk, sparse, dense in zip(chunks, embeddings["sparse"], embeddings["dense"]):
    # Assemble embedding vector, original text chunk, metadata.
    chunk_dict = {
        'chunk': chunk.page_content,
        'h1': chunk.metadata.get('h1', "")[:50],
        'h2': chunk.metadata.get('h2', "")[:50],
        'source': chunk.metadata.get('source', ""),
        'sparse_vector': sparse,
        'dense_vector': dense
    }
    dict_list.append(chunk_dict)

# TODO - Uncomment to inspect the first chunk and its metadata.
print(len(dict_list))
print(type(dict_list[0]), len(dict_list[0]))
pprint.pprint(dict_list[0])

304
<class 'dict'> 6
{'chunk': 'Why Milvus Docs Tutorials Tools Blog Community Stars0 Try Managed '
          'Milvus FREE Search Home v2.4.x About Milvus Get '
          'StartedPrerequisitesInstall MilvusInstall SDKsQuickstart Concepts '
          'User Guide Embeddings Administration Guide Tools Integrations '
          'Example Applications FAQs API reference Quickstart This guide '
          'explains how to connect to your Milvus cluster and performs CRUD '
          'operations in minutes Before you start You have installed Milvus '
          'standalone or Milvus cluster. You have installed preferred SDKs. '
          'You can',
 'dense_vector': array([-0.01666467,  0.05284622, -0.05246124, ..., -0.0182556 ,
        0.03670057, -0.00945159], dtype=float32),
 'h1': 'Why Milvus Docs Tutorials Tools Blog Community Sta',
 'h2': '',
 'source': 'https://milvus.io/docs/quickstart.md',
 'sparse_vector': <1x250002 sparse array of type '<class 'numpy.float32'>'
	with 63 stored elements i

In [14]:
# Insert data into the Milvus collection.
print("Start inserting entities")
start_time = time.time()
col.insert(dict_list)

end_time = time.time()
print(f"Milvus insert time for {len(dict_list)} vectors: ", end="")
print(f"{np.round(end_time - start_time, 2)} seconds")
col.flush()

Start inserting entities
Milvus insert time for 304 vectors: 0.19 seconds


## Aside - example Milvus collection API calls
https://milvus.io/docs/manage-collections.md#View-Collections

Below are some common API calls for checking a collection.
- `.num_entities`, flushes data and executes row count.
- `.describe_collection()`, gives details about the schema, index, collection.
- `.query()`, gives back selected data from the collection.

In [15]:
# Example Milvus Collection utility API calls.
# https://milvus.io/docs/manage-collections.md#View-Collections

# Count rows.
start_time = time.time()
print(f"Count rows: {col.num_entities}")
end_time = time.time()
print(f"timing: {np.round(end_time - start_time, 4)} seconds")
print()

# View collection info, incurs a call to .flush() first.
start_time = time.time()
pprint.pprint(mc.describe_collection(COLLECTION_NAME))
end_time = time.time()
print(f"timing: {np.round(end_time - start_time, 4)} seconds")
print()

# View rows. Careful, this can be a lot of output!
# OUTPUT_FIELDS = ["id", "h1", "h2", "source", "chunk"]
# res = mc.query( collection_name=COLLECTION_NAME, 
#                filter="id <= 449197422118227014", 
#                output_fields = OUTPUT_FIELDS, )
# pprint.pprint(res)

Count rows: 304
timing: 0.002 seconds

{'aliases': [],
 'auto_id': True,
 'collection_id': 449601743893922125,
 'collection_name': 'MilvusDocs',
 'consistency_level': 3,
 'description': '',
 'enable_dynamic_field': False,
 'fields': [{'auto_id': True,
             'description': '',
             'field_id': 100,
             'is_primary': True,
             'name': 'id',
             'params': {},
             'type': <DataType.INT64: 5>},
            {'description': '',
             'field_id': 101,
             'name': 'sparse_vector',
             'params': {},
             'type': <DataType.SPARSE_FLOAT_VECTOR: 104>},
            {'description': '',
             'field_id': 102,
             'name': 'dense_vector',
             'params': {'dim': 1024},
             'type': <DataType.FLOAT_VECTOR: 101>},
            {'description': '',
             'field_id': 103,
             'name': 'chunk',
             'params': {'max_length': 65535},
             'type': <DataType.VARCHAR: 21>

## Ask a question about your data

So far in this demo notebook: 
1. Your custom data has been mapped into a vector embedding space
2. Those vector embeddings have been saved into a vector database

Next, you can ask a question about your custom data!

💡 In LLM vocabulary:
> **Query** is the generic term for user questions.  
A query is a list of multiple individual questions, up to maybe 1000 different questions!

> **Question** usually refers to a single user question.  
In our example below, the user question is "What is AUTOINDEX in Milvus Client?"

> **Semantic Search** = very fast search of the entire knowledge base to find the `TOP_K` documentation chunks with the closest embeddings to the user's query.

💡 The same model should always be used for consistency for all the embeddings data and the query.

In [16]:
# Define a sample question about your data.
QUESTION1 = "What do the parameters for HNSW mean?"
QUESTION2 = "What are good default values for HNSW parameters with 25K vectors dim 1024?"
QUESTION3 = "What is the default AUTOINDEX distance metric in Milvus Client?"
QUESTION4 = "What does nlist mean in ivf_flat?"

# In case you want to ask all the questions at once.
QUERY = [QUESTION1, QUESTION2, QUESTION3, QUESTION4]

# Inspect the length of one question.
QUERY_LENGTH = len(QUESTION2)
print(f"query length: {QUERY_LENGTH}")

query length: 75


In [17]:
# SELECT A PARTICULAR QUESTION TO ASK.

SAMPLE_QUESTION = QUESTION1

## Execute a vector search

Search Milvus using [PyMilvus API](https://milvus.io/docs/search.md).

💡 By their nature, vector searches are "semantic" searches.  For example, if you were to search for "leaky faucet": 
> **Traditional Key-word Search** - either or both words "leaky", "faucet" would have to match some text in order to return a web page or link text to the document.

> **Semantic search** - results containing words "drippy" "taps" would be returned as well because these words mean the same thing even though they are different words.

In [18]:
# STEP 7. RETRIEVE ANSWERS FROM YOUR DOCUMENTS STORED IN MILVUS OR ZILLIZ.

# Load the index into memory for search.
col.load()

# Embed the question using the same encoder.
query_embeddings = embedding_model([SAMPLE_QUESTION])
TOP_K = 2

# Return top k results with HNSW index.
SEARCH_PARAMS = dict({
    # Re-use index param for num_candidate_nearest_neighbors.
    "ef": INDEX_PARAMS['efConstruction']
    })

# Prepare the search requests for both vector fields
sparse_search_params = {"metric_type": "IP"}
sparse_req = AnnSearchRequest(
                query_embeddings["sparse"],
                "sparse_vector", sparse_search_params, limit=TOP_K)

dense_search_params = {"metric_type": "COSINE"}
dense_search_params.update(SEARCH_PARAMS)
dense_req = AnnSearchRequest(
                query_embeddings["dense"],
                "dense_vector", dense_search_params, limit=TOP_K)

# Define output fields to return.
OUTPUT_FIELDS = ["id", "h1", "h2", "source", "chunk"]

# Run semantic vector search using your query and the vector database.
start_time = time.time()
# Use the reranker.
results = col.hybrid_search([
            sparse_req, dense_req], rerank=RRFRanker(),
            limit=TOP_K, output_fields=OUTPUT_FIELDS)
# # No reranking use dense only.
# results = col.hybrid_search([
#             sparse_req, dense_req], rerank=WeightedRanker(0., 1.0),
#             limit=TOP_K, output_fields=OUTPUT_FIELDS)

elapsed_time = time.time() - start_time
print(f"Milvus Client search time for {len(dict_list)} vectors: {elapsed_time} seconds")

# Inspect search result.
print(f"type: {type(results[0])}, count: {len(results[0])}")

# Currently Milvus only support 1 query in the same hybrid search request, so
# we inspect res[0] directly. In future release Milvus will accept batch
# hybrid search queries in the same call.
results = results[0]

# Milvus Client search time for 304 vectors: 0.02100086212158203 seconds
# type: <class 'pymilvus.client.abstract.Hits'>, count: 2

Milvus Client search time for 304 vectors: 0.00729680061340332 seconds
type: <class 'pymilvus.client.abstract.Hits'>, count: 2


## Assemble and inspect the search result

The search result is in the variable `results[0]` consisting of top_k-count of objects of type `'pymilvus.client.abstract.Hits'`



In [19]:
# Assemble retrieved context and context metadata.
METADATA_FIELDS = [f for f in OUTPUT_FIELDS if f != 'chunk']

# Assemble retrieved ids, distances, contexts, sources, and metadata.
ids = []
distances = []
contexts = []
sources = []
metas = []
for i in range(len(results)):
    print(f"Retrieved result #{results[i].id}")
    ids.append(results[i].id)
    print(f"distance = {results[i].distance}")
    distances.append(results[i].distance)
    # print(f"Context: {results[i].entity.chunk[:150]}")
    contexts.append(results[i].entity.chunk)
    for j in METADATA_FIELDS:
        if hasattr(results[i].entity, j):
            meta_dict = {j: getattr(results[i].entity, j)}
            metas.append(meta_dict)
            if j == "source":
                # print(f"{j}: {getattr(results[i].entity, j)}")
                sources.append(getattr(results[i].entity, j))
    print()

# Keep results in a list of tuples.
formatted_results = list(zip(ids, distances, contexts, sources, metas)) 

# TODO - Uncomment to print the results.
pprint.pprint(formatted_results)


# Reranking: I only see differences in positions 7-10, when k=10.
# Rerank = True
# Retrieved result #449197422118225569
# distance = 0.032522473484277725

# Retrieved result #449197422118225570
# distance = 0.032522473484277725

# Retrieved result #449197422118225710
# distance = 0.03079839050769806

# Retrieved result #449197422118225553
# distance = 0.03077651560306549

# Retrieved result #449197422118225771
# distance = 0.03077651560306549

# Retrieved result #449197422118225565
# distance = 0.03030998818576336

# Retrieved result #449197422118225604
# distance = 0.01587301678955555

# Retrieved result #449197422118225631
# distance = 0.015384615398943424

# Retrieved result #449197422118225568

# Rerank = False
# Retrieved result #449197422118225569
# distance = 0.39207690954208374

# Retrieved result #449197422118225570
# distance = 0.36890196800231934

# Retrieved result #449197422118225710
# distance = 0.25794607400894165

# Retrieved result #449197422118225553
# distance = 0.25781089067459106

# Retrieved result #449197422118225771
# distance = 0.25772351026535034

# Retrieved result #449197422118225565
# distance = 0.2537841796875

# Retrieved result #449197422118225604
# distance = 0.2157345414161682

# Retrieved result #449197422118225575
# distance = 0.2065277099609375

# Retrieved result #449197422118225562


Retrieved result #449601743892518433
distance = 0.032522473484277725

Retrieved result #449601743892518434
distance = 0.032522473484277725

[(449601743892518433,
  0.032522473484277725,
  'this value can improve recall rate at the cost of increased search time. '
  '[1, 65535] 2 HNSW HNSW (Hierarchical Navigable Small World Graph) is a '
  'graph-based indexing algorithm. It builds a multi-layer navigation '
  'structure for an image according to certain rules. In this structure, the '
  'upper layers are more sparse and the distances between nodes are farther; '
  'the lower layers are denser and the distances between nodes are closer. The '
  'search starts from the uppermost layer, finds the node closest to the '
  'target',
  'https://milvus.io/docs/index.md',
  {'id': 449601743892518433}),
 (449601743892518434,
  0.032522473484277725,
  'layer, finds the node closest to the target in this layer, and then enters '
  'the next layer to begin another search. After multiple iterations

## Use an LLM to Generate a chat response to the user's question using the Retrieved Context.

Many different generative LLMs exist these days.  Check out the lmsys [leaderboard](https://chat.lmsys.org/?leaderboard).

In this notebook, we'll try these LLMs:
- The newly released open-source Llama 3 from Meta.
- The cheapest, paid model from Anthropic Claude3 Haiku.
- The standard in its price cateogory, gpt-3.5-turbo, from Openai.

In [20]:
# STEP 8. LLM-GENERATED ANSWER TO THE QUESTION, GROUNDED BY RETRIEVED CONTEXT.

# Separate all the context together by space.
# Lance Martin, LangChain, says put best contexts at end.
contexts_combined = ' '.join(reversed(contexts))
# Separate all the sources together by comma.
source_combined = ' '.join(reversed(sources))
print(f"Length long text to summarize: {len(contexts_combined)}")

# Define temperature for the LLM and random seed.
TEMPERATURE = 0.1
RANDOM_SEED = 415
FREQUENCY_PENALTY = 2

Length long text to summarize: 1017


# Try Llama3 with Ollama to generate a human-like chat response to the user's question

Follow the instructions to install ollama and pull a model.<br>
https://github.com/ollama/ollama

View details about which models are supported by ollama. <br>
https://ollama.com/library/llama3

That page says `ollama run llama3` will by default pull the latest "instruct" model, which is fine-tuned for chat/dialogue use cases.

The other kind of llama3 models are "pre-trained" base model. <br>
Example: ollama run llama3:text ollama run llama3:70b-text

**Format** `gguf` means the model runs on CPU.  gg = "Georgi Gerganov", creator of the C library model format ggml, which was recently changed to gguf.

**Quantization** (think of it like vector compaction) can lead to higher throughput at the expense of lower accuracy.  For the curious, quantization meanings can be found on: <br>
https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/tree/main.  

Below just listing the main quantization types.
- **q4_0**: Original quant method, 4-bit.
- **q4_k_m**: Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K
- **q5_0**: Higher accuracy, higher resource usage and slower inference.
- **q5_k_m**: Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K
- **q 6_k**: Uses Q8_K for all tensors
- **q8_0**: Almost indistinguishable from float16. High resource use and slow. Not recommended for most users.

In [21]:
# !python -m pip install ollama
import ollama

# Verify details which model you are running.
ollama_llama3 = ollama.list()['models'][0]

# Print the model details.
keys = ['format', 'parameter_size', 'quantization_level']
print(f"MODEL:{ollama.list()['models'][0]['name']}", end=", ")
for key in keys:
    print(f"{str.upper(key)}:{ollama.list()['models'][0]['details'].get(key, 'Key not found in dictionary')}", end=", ")
print(end="\n\n")

MODEL:llama3:latest, FORMAT:gguf, PARAMETER_SIZE:8B, QUANTIZATION_LEVEL:Q4_0, 



In [22]:
SYSTEM_PROMPT = f"""Given the provided Context, your task is to 
understand the content and accurately answer the question based 
on the information available in the context.  
Provide a complete, clear, concise, relevant response in fewer
than 4 sentences and cite the unique Sources.
Answer: The answer to the question.
Sources: {source_combined}
Context: {contexts_combined}
"""

In [23]:
# Send the question to llama 3 chat.
response = ollama.chat(
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT,},
        {"role": "user", "content": f"question: {SAMPLE_QUESTION}",}
    ],
    model='llama3',
    options={"temperature": TEMPERATURE, "seed": RANDOM_SEED,}
)
pprint.pprint(response['message']['content'].replace('\n', ' '))

('According to the provided context, the parameters for HNSW (Hierarchical '
 'Navigable Small World Graph) are:  * `M`: Maximum degree of nodes on each '
 'layer of the graph. This value can improve recall rate at the cost of '
 'increased search time. \t+ Range: (2, 2048) * `ef` (or `efConstruction` when '
 'building the index): Search range parameter. \t+ Range: [1, 65535]  These '
 'parameters allow you to control the trade-off between recall rate and search '
 'time when using HNSW for indexing and searching.')


In [24]:
# ('According to the provided context, the parameters for HNSW (Hierarchical '
#  'Navigable Small World Graph) are:  * `M`: Maximum degree of nodes on each '
#  'layer of the graph. This value can improve recall rate at the cost of '
#  'increased search time. \t+ Range: (2, 2048) * `ef` (or `efConstruction` when '
#  'building the index): Search range parameter. \t+ Range: [1, 65535]  These '
#  'parameters allow you to control the trade-off between recall rate and search '
#  'time when using HNSW for indexing and searching.')

## Use Anthropic to generate a human-like chat response to the user's question 

We've practiced retrieval for free on our own data using open-source LLMs.  <br>

Now let's make a call to the paid Claude3. [List of models](https://docs.anthropic.com/claude/docs/models-overview)
- Opus - most expensive
- Sonnet
- Haiku - least expensive!

Prompt engineering tutorials
- [Interactive](https://docs.google.com/spreadsheets/d/19jzLgRruG9kjUQNKtCg1ZjdD6l6weA6qRXG5zLIAhC8/edit#gid=150872633)
- [Static](https://docs.google.com/spreadsheets/d/1jIxjzUWG-6xBVIa2ay6yDpLyeuOh_hR_ZB75a47KX_E/edit#gid=869808629)

In [25]:
SYSTEM_PROMPT = f"""Use the Context below to answer the user's question. 
Be clear, factual, complete, concise.
If the answer is not in the Context, say "I don't know". 
Otherwise answer with fewer than 4 sentences and cite the unique sources.
Context: {contexts_combined}
Sources: {source_combined}

Answer with 2 parts: the answer and the source citations.
Answer: The answer to the question.
Sources: unique url sources
"""

In [26]:
# # !python -m pip install anthropic
# import anthropic

# ANTHROPIC_API_KEY=os.environ.get("ANTHROPIC_API_KEY")

# # # Model names
# # claude-3-opus-20240229
# # claude-3-sonnet-20240229
# # claude-3-haiku-20240307
# CLAUDE_MODEL = "claude-3-haiku-20240307"
# print(f"Model: {CLAUDE_MODEL}")
# print()

# client = anthropic.Anthropic(
#     # defaults to os.environ.get("ANTHROPIC_API_KEY")
#     api_key=ANTHROPIC_API_KEY,
# )

# # Print the question and answer along with grounding sources and citations.
# print(f"Question: {SAMPLE_QUESTION}")

# # CAREFUL!! THIS COSTS MONEY!!
# message = client.messages.create(
#     model=CLAUDE_MODEL,
#     max_tokens=1000,
#     temperature=0.0,
#     system=SYSTEM_PROMPT,
#     messages=[
#         {"role": "user", "content": SAMPLE_QUESTION}
#     ]
# )
# print("Answer:")
# pprint.pprint(message.content[0].text.replace('\n', ' '))

In [27]:
# # model="claude-3-haiku-20240307"

# # Question: What do the parameters for HNSW mean?
# Answer:
# ('The parameters for HNSW (Hierarchical Navigable Small World Graph) are:  1. '
#  'M: This is the maximum degree of the nodes in the graph. It controls the '
#  'sparsity of the upper layers and the density of the lower layers. The range '
#  'for M is (2, 2048).  2. efConstruction: This parameter specifies the search '
#  'range when building the index. It affects the recall rate and search time - '
#  'a higher value can improve recall at the cost of increased search time.  3. '
#  'ef: This parameter specifies the search range when searching for targets. '
#  'Similar to efConstruction, a higher value can improve recall but increase '
#  'search time.  Sources: [1] https://milvus.io/docs/index.md')

In [28]:
# # model="claude-3-sonnet-20240229"

# # Question: What do the parameters for HNSW mean?
# # Answer:
# ('The parameters M and ef/efConstruction control the behavior of the HNSW '
#  '(Hierarchical Navigable Small World) algorithm used for indexing and '
#  'searching.  M specifies the maximum number of connections (edges) that each '
#  'node in the HNSW graph can have. A higher M value allows more connections, '
#  'which can improve recall rate (finding more relevant results) but increases '
#  'search time.  ef and efConstruction determine how many nodes in each layer '
#  'of the HNSW graph should be explored during searching and index construction '
#  'respectively. Higher values increase the search range and can improve '
#  'accuracy, but also increase computation time.  Sources: [1] '
#  'https://milvus.io/docs/index.md [2] https://milvus.io/docs/index.md')

In [29]:
# # model="claude-3-opus-20240229"

# # Question: What do the parameters for HNSW mean?
# # Answer:
# ('According to the context, the HNSW algorithm has two key parameters:  1. M: '
#  'The maximum degree of the node on each layer of the graph, which can be set '
#  'between 2 and 2048. [1]  2. efConstruction (when building index) or ef (when '
#  'searching targets): These parameters specify the search range to improve '
#  'performance. [1]  Grounding sources: [1] https://milvus.io/docs/index.md')

<div>
<img src="../../../images/anthropic_claude3.png" width="80%"/>
</div>

## Try MistralAI's Mixtral 8x22B-Instruct-v0.1 to generate a human-like chat response to the user's question 

This time ollama's version requires 48GB RAM. If you have big enough compute, run the command:
> ollama run mixtral

Since my laptop is a M2 with only 16GB RAM, I decided to **run Mixtral using Anyscale Endpoints**. Instructions to install. <br>
> https://github.com/simonw/llm-anyscale-endpoints

To get back to **Anyscale Endpoints** anytime, open the playground.<br>
https://console.anyscale.com/v2/playground

In [30]:
SYSTEM_PROMPT = f"""Use the Context below to answer the user's question. 
Be complete, clear, concise, relevant.
If the answer is not in the Context, say "I don't know". 
Otherwise answer with fewer than 4 sentences 
and cite only unique grounding sources.
Answer: The answer to the question.
Non-unique grounding sources: {source_combined}
Context: {contexts_combined}
"""
# print(SYSTEM_PROMPT)

In [31]:
# SAMPLE_QUESTION

In [32]:
# # model="Mixtral 8x22B-Instruct-v0.1"

# # Question: What do the parameters for HNSW mean?
# # Answer:
# The parameters for HNSW (Hierarchical Navigable Small World Graph) include M, which represents the 
# maximum degree of the node in the HNSW graph, ranging from 2 to 2048. A higher value of M can 
# improve recall rate at the cost of increased search time. Additionally, efConstruction (used 
# during index building) and ef (used during target search) parameters are used to specify a search range.
# Reference(s):
# https://milvus.io/docs/index.md

<div>
<img src="../../../images/mistral_mixtral.png" width="80%"/>
</div>

## Try OpenAI to generate a human-like chat response to the user's question 

We've practiced retrieval for free on our own data using open-source LLMs.  <br>

Now let's make a call to the paid OpenAI GPT.

💡 Note: For use cases that need to always be factually grounded, use very low temperature values while more creative tasks can benefit from higher temperatures.

In [33]:
SYSTEM_PROMPT = f"""Use the Context below to answer the user's question. 
Be complete, clear, concise, relevant.
If the answer is not in the Context, say "I don't know". 
Otherwise answer with fewer than 4 sentences and cite the grounding sources.
Answer: The answer to the question.
Grounding sources: {source_combined}
Context: {contexts_combined}
"""

In [34]:
# CAREFUL!! THIS COSTS MONEY!!
import openai, pprint
from openai import OpenAI

# 1. Define the generation llm model to use.
# https://openai.com/blog/new-embedding-models-and-api-updates
# Customers using the pinned gpt-3.5-turbo model alias will be automatically upgraded to gpt-3.5-turbo-0125 two weeks after this model launches.
LLM_NAME = "gpt-3.5-turbo"

# 2. Get your API key: https://platform.openai.com/api-keys
# 3. Save your api key in env variable.
# https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety
openai_client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# 4. Generate response using the OpenAI API.
response = openai_client.chat.completions.create(
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT,},
        {"role": "user", "content": f"question: {SAMPLE_QUESTION}",}
    ],
    model=LLM_NAME,
    temperature=TEMPERATURE,
    seed=RANDOM_SEED,
    frequency_penalty=2,
)

# Print the question and answer along with grounding sources and citations.
print(f"Question: {SAMPLE_QUESTION}")

# 5. Print all answers in the response.
for i, choice in enumerate(response.choices, 1):
    pprint.pprint(f"Answer: {choice.message.content}")
    print("\n")

# Question1: What do the parameters for HNSW mean?
# Answer:  Looks perfect!
# Best answer:  M: maximum degree of nodes in a layer of the graph. 
# efConstruction: number of nearest neighbors to consider when connecting nodes in the graph.
# ef: number of nearest neighbors to consider when searching for similar vectors. 

# Question2: What are good default values for HNSW parameters with 25K vectors dim 1024?
# Answer: M=16, efConstruction=500, and ef=64
# Best answer:  M=16, efConstruction=32, ef=32

# Question3: what is the default distance metric used in AUTOINDEX in Milvus?
# Answer: L2 
# Trick answer:  IP inner product, not yet updated in documentation still says L2.

# Question4: What does nlist mean in ivf_flat?
# 'Answer: In IVF_FLAT, nlist refers to the number of cluster units that divide '
#  'a vector space. When using the default value of 16384 for nlist in Milvus, '
#  "distances between the target vector and all 16384 clusters' centers are "
#  'compared to find the nearest clusters for further comparison with vectors '
#  'within those selected clusters. This parameter influences how clustering is '
#  'performed and affects search efficiency in Milvus.\n'
#  'Sources: https://milvus.io/docs/index.md')

Question: What do the parameters for HNSW mean?
('Answer: The parameters for HNSW are as follows:\n'
 '- M: Maximum degree of the node (2, 2048), which limits the maximum '
 'connections a node can have in each layer.\n'
 '- efConstruction: Used during index building to specify a search range, '
 'improving recall rate at the cost of increased search time.\n'
 '- ef: Used when searching targets to specify a search range. It determines '
 'how many nodes are visited in each layer during the search process.')




In [35]:
# model="gpt-3.5-turbo"

# Question: What do the parameters for HNSW mean?
# ('Answer: The parameters for HNSW are M, which is the maximum degree of nodes '
#  'on each layer of the graph (ranging from 2 to 2048), and efConstruction '
#  '(used during index building) or ef (used during target searching) to specify '
#  'a search range. The parameter M can improve recall rate at the expense of '
#  'increased search time, while ef controls how exhaustive the search will be '
#  'within a certain distance. These parameters help optimize performance in '
#  'navigating through multi-layer structures efficiently.\n'
#  'Grounding sources: https://milvus.io/docs/index.md')

## Use Ragas to evaluate RAG pipeline

Ragas is an open source project for evaluating RAG components.  [Paper](https://arxiv.org/abs/2309.15217), [Code](https://docs.ragas.io/en/stable/getstarted/index.html), [Docs](https://docs.ragas.io/en/stable/getstarted/index.html), [Intro blog](https://medium.com/towards-data-science/rag-evaluation-using-ragas-4645a4c6c477).

<div>
<img src="../../../images/ragas_eval_image.png" width="80%"/>
</div>

**Please note that RAGAS can use a large amount of OpenAI api token consumption.** <br> 

Read through this notebook carefully and pay attention to the number of questions and metrics you want to evaluate.



In [36]:
import os, sys
import pandas as pd
import numpy as np
import ragas, datasets
from langchain_community.embeddings import HuggingFaceEmbeddings
from ragas.embeddings import LangchainEmbeddingsWrapper

# Import custom functions for evaluation.
sys.path.append("../../Integration")  
import eval_ragas as _eval_ragas

# Import the evaluation metrics.
from ragas.metrics import (
    context_recall, 
    context_precision, 
    faithfulness, 
    answer_relevancy, 
    answer_similarity,
    answer_correctness
    )

# Get the current working directory.
cwd = os.getcwd()
relative_path = '/../../Evaluation/data/ground_truth_answers.csv'
file_path = cwd + relative_path
# print(f"file_path: {file_path}")

# Read ground truth answers from file.
eval_df = pd.read_csv(file_path, header=0, skip_blank_lines=True)
display(eval_df.head())

,Question,ground_truth_answer,Sources,Custom_RAG_context,Simple,Custom_RAG_answer,llama3_answer,anthropic_claud3_haiku_answer,mixtral_8x22b_instruct
0,What do the parameters for HNSW mean?,"# M: maximum degree, or number of connections ...",https://milvus.io/docs/index.md,HNSW (Hierarchical Navigable Small World Graph...,HNSW (Hierarchical Navigable Small World Graph...,The parameters for HNSW are as follows:\n- M: ...,The parameters for HNSW (Hierarchical Navigabl...,The parameters for HNSW (Hierarchical Navigabl...,The parameter M in HNSW represents the maximum...
1,What are good default values for HNSW paramete...,"M=16, efConstruction=32, ef=32","https://milvus.io/docs/index.md, https://milvu...",parameters vary with Milvus distribution. Sele...,nbits [Optional] Number of bits in which each ...,"M=16, efConstruction=500, and ef=64",HNSW (Hierarchical Navigable Small World) inde...,I don't know. The context provided does not co...,"Based on the provided Context, good default va..."
2,What does nlist vs nprobe mean in ivf_flat?,# nlist: controls how the vector data is part...,https://milvus.io/docs/index.md,FAQ What is the difference between FLAT index...,index? IVF_FLAT index divides a vector space i...,"In IVF_FLAT, nlist refers to the number of clu...",`nlist` refers to the number of cluster units ...,The nlist parameter in the IVF_FLAT index in M...,"nlist, which stands for ""number of cluster uni..."
3,What is the default AUTOINDEX distance metric ...,"Trick answer: IP inner product, not yet updat...",https://milvus.io/docs/index.md,FAQs API reference Similarity Metrics In Mi...,FAQs API reference Similarity Metrics In Mi...,The default distance metric for the AUTOINDEX ...,"According to the provided context, the answer ...",The default AUTOINDEX distance metric in Milvu...,The default distance metric for AUTOINDEX in M...


In [40]:
# TODO: Make this test harness easier to use.
##########################################
# EVALUATE_WHAT = 'ANSWERS' or 'CONTEXTS'
##########################################
EVALUATE_WHAT = 'ANSWERS' 
# EVALUATE_WHAT = 'CONTEXTS' 

# Possible LLM model choices to evaluate:
# 1. openai gpt-3.5-turbo = 'Custom_RAG_answer'
# 2. llama3_answer
# 3. anthropic_claud3_haiku_answer
LLM_TO_EVALUATE = 'Custom_RAG_answer'
# LLM_TO_EVALUATE = 'llama3_answer'
# LLM_TO_EVALUATE = 'mixtral_8x22b_instruct'
# LLM_TO_EVALUATE = 'anthropic_claud3_haiku_answer'

# Possible chunking strategies to evaluate:
# 1. recursivetextsplitter = 'Simple'
# 2. htmlsplitter = 'Custom_RAG_context'
# 3. small_big_splitter
# CONTEXT_TO_EVALUATE='Simple'
CONTEXT_TO_EVALUATE='Custom_RAG_context'

if EVALUATE_WHAT == 'ANSWERS':
    CONTEXT_TO_EVALUATE='Custom_RAG_context'
    eval_metrics=[
        answer_relevancy,
        answer_similarity,
        answer_correctness,
        faithfulness,
        ]
    metrics = ['answer_relevancy', 'answer_similarity', 'answer_correctness', 'faithfulness']

elif EVALUATE_WHAT == 'CONTEXTS':
    LLM_TO_EVALUATE = 'Custom_RAG_answer'
    eval_metrics=[
        context_recall, 
        context_precision,
        faithfulness,
        ]
    metrics = ['context_recall', 'context_precision', 'faithfulness']
    
# Change the default the llm-as-critic.
LLM_NAME = "gpt-3.5-turbo"
ragas_llm = ragas.llms.llm_factory(model=LLM_NAME)

# Change the default embeddings to HuggingFace models.
EMB_NAME = "BAAI/bge-large-en-v1.5"
lc_embeddings = HuggingFaceEmbeddings(model_name=EMB_NAME)
ragas_emb = LangchainEmbeddingsWrapper(embeddings=lc_embeddings)

# Change each metric.
for metric in metrics:
    globals()[metric].llm = ragas_llm
    globals()[metric].embeddings = ragas_emb

# Execute the evaluation.
ragas_result, score = _eval_ragas.evaluate_ragas_model(
    eval_df, eval_metrics, LLM_TO_EVALUATE, 
    CONTEXT_TO_EVALUATE, EVALUATE_WHAT)

# Display the results.
print(f"Using {eval_df.shape[0]} eval questions, Mean Score = {score}")
display(ragas_result.head())

Evaluating:   0%|          | 0/16 [00:00<?, ?it/s]

No statements were generated from the answer.


LLM to evaluate: anthropic_claud3_haiku_answer
Using 4 eval questions, Mean Score = 0.6172


,question,contexts,answer,ground_truth,answer_relevancy,answer_similarity,answer_correctness,faithfulness,avg_answer_score
0,What do the parameters for HNSW mean?,[HNSW (Hierarchical Navigable Small World Grap...,The parameters for HNSW (Hierarchical Navigabl...,"# M: maximum degree, or number of connections ...",0.749758,0.757881,0.689470,1.0,0.732370
1,What are good default values for HNSW paramete...,[parameters vary with Milvus distribution. Sel...,I don't know. The context provided does not co...,"M=16, efConstruction=32, ef=32",0.000000,0.603921,0.150980,0.0,0.251634
2,What does nlist vs nprobe mean in ivf_flat?,[FAQ What is the difference between FLAT inde...,The nlist parameter in the IVF_FLAT index in M...,# nlist: controls how the vector data is part...,0.860034,0.769326,0.692331,0.4,0.773897
3,What is the default AUTOINDEX distance metric ...,[FAQs API reference Similarity Metrics In M...,The default AUTOINDEX distance metric in Milvu...,"Trick answer: IP inner product, not yet updat...",0.968280,0.631614,0.532904,1.0,0.710933


In [ ]:
####################################################
# Avg Context Precision htmlsplitter score = 0.54 (200% improvement)
# Avg Context Precision simple score = 0.17
####################################################

####################################################
# Avg openai gpt-3.5-turbo score = 0.7365 (10% improvement)
# Avg mistralai mixtral_8x22b_instruct score = 0.7272 (9% improvement)
# Avg llama3 answer score = 0.667 (8% improvement)
# Avg anthropic_claud3_haiku_answer score = 0.6172
####################################################

In [41]:
# Drop collection
utility.drop_collection(COLLECTION_NAME)

In [42]:
# Props to Sebastian Raschka for this handy watermark.
# !pip install watermark

%load_ext watermark
%watermark -a 'Christy Bergman' -v -p torch,datasets,pymilvus,langchain,ollama,anthropic,openai,ragas --conda

Author: Christy Bergman

Python implementation: CPython
Python version       : 3.11.8
IPython version      : 8.22.2

torch    : 2.2.2
datasets : 2.19.0
pymilvus : 2.4.1
langchain: 0.1.16
ollama   : 0.1.8
anthropic: 0.25.6
openai   : 1.14.3
ragas    : 0.1.7

conda environment: py311-unum

